In [27]:
import pandas as pd
import os
from google import genai
from docx import Document

In [28]:
#Cambiar directorio de trabajo
os.chdir(r"H:/Mi unidad\pgd\informacion_corto_plazo")

In [29]:
##Leer matriz de consolidación
matriz=pd.read_excel("matriz_consolidacion.xlsx", usecols=range(8))
#Leer credenciales de google

with open('C:/cred_google_gemini.txt', 'r') as file:
    llave = file.read()

In [38]:
# 2) Función que arma el “documento” por grupo
def documento_por_grupo(g, salto="\n\n"):
    diag = salto.join(
        f"Diagnóstico: {t.strip()}"
        for t in g.loc[g.variable.str.contains("Diagnóstico", na=False), "texto"]
    )
    obj  = salto.join(
        f"Objetivo: {t.strip()}"
        for t in g.loc[g.variable.str.contains("Objetivo", na=False), "texto"]
    )
    est  = salto.join(
        f"Estrategia: {t.strip()}"
        for t in g.loc[g.variable.str.contains("Estrategia", na=False), "texto"]
    )
    return diag, obj, est

In [39]:
grupos = (
    matriz.groupby(["linea", "eje", "tema", "documento"], dropna=False)
          .apply(documento_por_grupo)
)

print(f"Se generarán {len(grupos)} documentos Word")
grupos

Se generarán 5 documentos Word


C:\Users\claud\AppData\Local\Temp\ipykernel_24908\1336391203.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(documento_por_grupo)


linea                                       eje                                                                                                  tema                 documento                                                           
Plena de derechos de infancia y juventudes  Educación, cultura, disfrute, salud y cuidados que promuevan el libre desarrollo de la personalidad  Cultura              Programa provisional de gobierno del Gobierno de la Ciudad de México    (Diagnóstico: La capital concentra la mayor pr...
                                                                                                                                                 Educación            Programa provisional de gobierno del Gobierno de la Ciudad de México    (Diagnóstico: Datos de la Secretaría de Educac...
                                            Trabajo y vivienda, para juventudes                                                                  Grupos prioritarios  Programa provisional de

In [32]:
# 4. Crear carpeta de salida si no existe
if not os.path.exists("resumenes_docx"):
    os.makedirs("resumenes_docx")

In [33]:
client=genai.Client(api_key=llave)

In [44]:
for clave, (diag, obj, est) in grupos.items():
    linea, eje, tema, documento = clave

    # --- Prompt con prefijos ya incluidos -----------------------------------
    prompt = (
        "Redacta un documento extenso en donde se utilice la siguiente información:\n\n"
        f"{diag}\n\n{obj}\n\n{est}"
        "No redactes en primera persona, ni uses el pronombre 'yo' Y modifica marginalmente la redacción para que tenga coherencia y no sea íntegra.\n"
        "La estructura del documento debe ser la siguiente:\n"
        "Diagnóstico: [texto del diagnóstico]\n" \
        "Objetivos de corto plazo: [texto del objetivo]. No establezcas temporalidad en el título\n" \
        "Estrategias de corto plazo: [texto de la estrategia]\n\n"
    )

    resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )
    resumen = resp.candidates[0].content.parts[0].text.strip()


    # --- Crear DOCX ----------------------------------------------------------
    doc = Document()
    doc.add_heading(f"{linea} – {tema}", level=1)
    doc.add_paragraph(f"Eje: {eje}")
    #doc.add_paragraph(f"Documento fuente: {documento}")
    doc.add_heading("Documento", level=2)
    doc.add_paragraph(resumen)

    # Anexo con texto completo
    doc.add_page_break()
    doc.add_heading("Anexo: texto íntegro", level=2)
    if diag: doc.add_paragraph(diag)
    if obj:  doc.add_paragraph("\n" + obj)
    if est:  doc.add_paragraph("\n" + est)

    # Guardar
    nombre = f"{linea[:30]}_{tema[:20]}.docx".replace(" ", "_")
    ruta = os.path.join("resumenes_docx", nombre)
    doc.save(ruta)
    print("✓ Guardado:", ruta)

✓ Guardado: resumenes_docx\Plena_de_derechos_de_infancia__Cultura.docx
✓ Guardado: resumenes_docx\Plena_de_derechos_de_infancia__Educación.docx
✓ Guardado: resumenes_docx\Plena_de_derechos_de_infancia__Grupos_prioritarios.docx
✓ Guardado: resumenes_docx\Plena_de_derechos_de_infancia__Trabajo.docx
✓ Guardado: resumenes_docx\Plena_de_derechos_de_infancia__Vivienda.docx
